<a href="https://colab.research.google.com/github/ssv273/Neural_Univesity/blob/main/hw_18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -q kaggle
from google.colab import files 
files.upload()
! mkdir ~/.kaggle 
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d dataturks/resume-entities-for-ner
! mkdir Data
! unzip resume-entities-for-ner.zip -d Data

Saving kaggle.json to kaggle.json
  0% 0.00/323k [00:00<?, ?B/s]
100% 323k/323k [00:00<00:00, 44.4MB/s]
Archive:  resume-entities-for-ner.zip
  inflating: Data/Entity Recognition in Resumes.json  


In [2]:
import json
import re
import time

In [3]:
# JSON formatting functions
def convert_dataturks_to_spacy(dataturks_JSON_FilePath):
    training_data = []
    lines=[]
    with open(dataturks_JSON_FilePath, 'r') as f:
        lines = f.readlines()

    for line in lines:
        data = json.loads(line)
        text = data['content'].replace("\n", " ")
        entities = []
        data_annotations = data['annotation']
        if data_annotations is not None:
            for annotation in data_annotations:
                #only a single point in text annotation.
                point = annotation['points'][0]
                labels = annotation['label']
                # handle both list of labels or a single label.
                if not isinstance(labels, list):
                    labels = [labels]

                for label in labels:
                    point_start = point['start']
                    point_end = point['end']
                    point_text = point['text']

                    lstrip_diff = len(point_text) - len(point_text.lstrip())
                    rstrip_diff = len(point_text) - len(point_text.rstrip())
                    if lstrip_diff != 0:
                        point_start = point_start + lstrip_diff
                    if rstrip_diff != 0:
                        point_end = point_end - rstrip_diff
                    entities.append((point_start, point_end + 1 , label))
        training_data.append((text, {"entities" : entities}))
    return training_data

def trim_entity_spans(data: list) -> list:
    """Removes leading and trailing white spaces from entity spans.

    Args:
        data (list): The data to be cleaned in spaCy JSON format.

    Returns:
        list: The cleaned data.
    """
    invalid_span_tokens = re.compile(r'\s')

    cleaned_data = []
    for text, annotations in data:
        entities = annotations['entities']
        valid_entities = []
        for start, end, label in entities:
            valid_start = start
            valid_end = end
            while valid_start < len(text) and invalid_span_tokens.match(
                    text[valid_start]):
                valid_start += 1
            while valid_end > 1 and invalid_span_tokens.match(
                    text[valid_end - 1]):
                valid_end -= 1
            valid_entities.append([valid_start, valid_end, label])
        cleaned_data.append([text, {'entities': valid_entities}])
    return cleaned_data

In [4]:
data = trim_entity_spans(convert_dataturks_to_spacy("Data/Entity Recognition in Resumes.json"))
data[0]

["Abhishek Jha Application Development Associate - Accenture  Bengaluru, Karnataka - Email me on Indeed: indeed.com/r/Abhishek-Jha/10e7a8cb732bc43a  • To work for an organization which provides me the opportunity to improve my skills and knowledge for my individual and company's growth in best possible ways.  Willing to relocate to: Bangalore, Karnataka  WORK EXPERIENCE  Application Development Associate  Accenture -  November 2017 to Present  Role: Currently working on Chat-bot. Developing Backend Oracle PeopleSoft Queries for the Bot which will be triggered based on given input. Also, Training the bot for different possible utterances (Both positive and negative), which will be given as input by the user.  EDUCATION  B.E in Information science and engineering  B.v.b college of engineering and technology -  Hubli, Karnataka  August 2013 to June 2017  12th in Mathematics  Woodbine modern school  April 2011 to March 2013  10th  Kendriya Vidyalaya  April 2001 to March 2011  SKILLS  C (Le

In [5]:
data[0][0][0:12]

'Abhishek Jha'

In [6]:
len(data)

220

In [7]:
entity_dict = {
    'Name': 's0', 
    'College Name': 's1',
    'Degree': 's2',
    'Graduation Year': 's3',
    'Years of Experience': 's4',
    'Companies worked at': 's5',
    'Designation': 's6',
    'Skills': 's7',
    'Location': 's8',
    'Email Address': 's9',
    'UNKNOWN':'s10'
}

In [8]:
data[0][1]['entities'][::-1]

[[0, 12, 'Name'],
 [13, 46, 'Designation'],
 [49, 58, 'Companies worked at'],
 [60, 69, 'Location'],
 [95, 145, 'Email Address'],
 [372, 405, 'Designation'],
 [407, 416, 'Companies worked at'],
 [727, 769, 'Designation'],
 [771, 814, 'College Name'],
 [856, 860, 'Graduation Year'],
 [883, 905, 'College Name'],
 [939, 957, 'College Name'],
 [993, 1154, 'Skills'],
 [1296, 1622, 'Skills']]

In [9]:
data[0][0]

"Abhishek Jha Application Development Associate - Accenture  Bengaluru, Karnataka - Email me on Indeed: indeed.com/r/Abhishek-Jha/10e7a8cb732bc43a  • To work for an organization which provides me the opportunity to improve my skills and knowledge for my individual and company's growth in best possible ways.  Willing to relocate to: Bangalore, Karnataka  WORK EXPERIENCE  Application Development Associate  Accenture -  November 2017 to Present  Role: Currently working on Chat-bot. Developing Backend Oracle PeopleSoft Queries for the Bot which will be triggered based on given input. Also, Training the bot for different possible utterances (Both positive and negative), which will be given as input by the user.  EDUCATION  B.E in Information science and engineering  B.v.b college of engineering and technology -  Hubli, Karnataka  August 2013 to June 2017  12th in Mathematics  Woodbine modern school  April 2011 to March 2013  10th  Kendriya Vidyalaya  April 2001 to March 2011  SKILLS  C (Les

In [10]:
example = data[0][0]

In [11]:
def set_tags(string, entities):
    other_symbols_index = 0
    new_string = ''
    for i in entities[::-1]:
        # print("Левый текст", example[other_symbols_index:i[0]])
        # print(example[i[0]:i[1]])
        new_string = new_string + " " + string[other_symbols_index:i[0]] + " " + "<" + entity_dict[i[2]] + ">" + string[i[0]:i[1]] + "</" + entity_dict[i[2]] + ">"
        other_symbols_index = i[1]
        # print("-"* 20)

    return new_string

In [12]:
set_tags(data[0][0], data[0][1]['entities'])

"  <s0>Abhishek Jha</s0>   <s6>Application Development Associate</s6>  -  <s5>Accenture</s5>    <s8>Bengaluru</s8> , Karnataka - Email me on  <s9>Indeed: indeed.com/r/Abhishek-Jha/10e7a8cb732bc43a</s9>   • To work for an organization which provides me the opportunity to improve my skills and knowledge for my individual and company's growth in best possible ways.  Willing to relocate to: Bangalore, Karnataka  WORK EXPERIENCE   <s6>Application Development Associate</s6>    <s5>Accenture</s5>  -  November 2017 to Present  Role: Currently working on Chat-bot. Developing Backend Oracle PeopleSoft Queries for the Bot which will be triggered based on given input. Also, Training the bot for different possible utterances (Both positive and negative), which will be given as input by the user.  EDUCATION   <s6>B.E in Information science and engineering</s6>    <s1>B.v.b college of engineering and technology</s1>  -  Hubli, Karnataka  August 2013 to June  <s3>2017</s3>   12th in Mathematics   <s1>

In [13]:
new_texts = []
for i in range(len(data)):
    new_texts.append(set_tags(data[i][0], data[i][1]['entities']))

In [14]:
new_texts[10:11]

['  <s0>Asish Ratha</s0>   <s6>Subject matter Expert</s6>  -  <s5>Accenture</s5>    <s8>Chennai</s8> , Tamil Nadu - Email me on Indeed:  <s8>indeed.com/r/Asish-Ratha/853988e0e0e236a3</s8>   WORK EXPERIENCE   <s6>Subject matter Expert</s6>    <s5>Accenture</s5>  -  March 2012 to Present  Subject matter expert  EDUCATION   <s1>Berhampur university</s1> ,  <s1>Khallikote autonomous college</s1>  -  Brahmapur, Orissa  SKILLS   <s7>Invoice (5 years), posting. (5 years), TRAINING (4 years)  ADDITIONAL INFORMATION  SKILLS Invoice processing, Team handling, new joiners training.sap posting,vendor call attend and resolve the issue,meet SLA tat,working with client tool.</s7>']

In [15]:
# Считываем текст из файла и удаляем пунктуационные знаки препинания и еще дополнительные ненужные нам знаки
def readText(text):
  # Определяем, какие знаки будут удалены
  delSymbols = ['\n', "\t", "\ufeff", ".", "_", "-", ",", "!", "?", "–", "(", ")", "«", "»", "№", ";",'•','%']

  for dS in delSymbols: # Каждый знак из нашего списка будет удалён из списка
    text = text.replace(dS, " ") # Удаляем знак, посредством замены этого знака на пробел

  # Выискиваем дополнительные знаки в текстах посредством паттернов и определяем на что их заменять
  text = re.sub("[.]", " ", text)
  text = re.sub(":", " ", text)
  text = re.sub("<", " <", text)
  text = re.sub(">", "> ", text)

  # split  метод разделит текст на последовательность слов при помощи пробелов
  # (а их может быть в одном месте очень много после наших удалений знаков), которые будет являться разделителями.
  # Используя join метод, мы обратно соберём последовательности слов в текст
  text = ' '.join(text.split()) 

  text = text.lower() # Конвертируем текст к нижнему регистру
  return text # Возвращаем текст



In [16]:
readText(new_texts[0])

"<s0> abhishek jha </s0> <s6> application development associate </s6> <s5> accenture </s5> <s8> bengaluru </s8> karnataka email me on <s9> indeed indeed com/r/abhishek jha/10e7a8cb732bc43a </s9> to work for an organization which provides me the opportunity to improve my skills and knowledge for my individual and company's growth in best possible ways willing to relocate to bangalore karnataka work experience <s6> application development associate </s6> <s5> accenture </s5> november 2017 to present role currently working on chat bot developing backend oracle peoplesoft queries for the bot which will be triggered based on given input also training the bot for different possible utterances both positive and negative which will be given as input by the user education <s6> b e in information science and engineering </s6> <s1> b v b college of engineering and technology </s1> hubli karnataka august 2013 to june <s3> 2017 </s3> 12th in mathematics <s1> woodbine modern school </s1> april 2011 

In [17]:
new_texts[0]

"  <s0>Abhishek Jha</s0>   <s6>Application Development Associate</s6>  -  <s5>Accenture</s5>    <s8>Bengaluru</s8> , Karnataka - Email me on  <s9>Indeed: indeed.com/r/Abhishek-Jha/10e7a8cb732bc43a</s9>   • To work for an organization which provides me the opportunity to improve my skills and knowledge for my individual and company's growth in best possible ways.  Willing to relocate to: Bangalore, Karnataka  WORK EXPERIENCE   <s6>Application Development Associate</s6>    <s5>Accenture</s5>  -  November 2017 to Present  Role: Currently working on Chat-bot. Developing Backend Oracle PeopleSoft Queries for the Bot which will be triggered based on given input. Also, Training the bot for different possible utterances (Both positive and negative), which will be given as input by the user.  EDUCATION   <s6>B.E in Information science and engineering</s6>    <s1>B.v.b college of engineering and technology</s1>  -  Hubli, Karnataka  August 2013 to June  <s3>2017</s3>   12th in Mathematics   <s1>

In [18]:
from nltk.corpus import wordnet
import nltk

In [19]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [20]:
# Конвертируем исходный текст в лист слов с начальной формой 
def text2Words(text):
  lemmatizer = nltk.stem.wordnet.WordNetLemmatizer() # Инициализируем инструмент для работы с морфемами и более
  words = text.split(' ') #  Разделяем текст по средством пробелов
  docs = [lemmatizer.lemmatize(word, wordnet.VERB) for word in words]
#   docs = [morph.parse(word)[0].normal_form for word in words] # Превращаем каждое слово в элемент списка
  return docs # Возвращаем полученный документ

In [21]:
docs_full = [] # Лист будет содержать все договоры как списки документов
curTime = time.time() # Запоминаем текущее время
for i in range(len(new_texts)): # Проходимся по каждому договору
  docs_full.append(text2Words(readText(new_texts[i]))) # Превращаем договор в лист слов и добавляем его в docs_full
print('Превращение заняло: ', round(time.time() - curTime, 2), 's.')

Превращение заняло:  1.91 s.


In [22]:
docs_full[0][:10]

['<s0>',
 'abhishek',
 'jha',
 '</s0>',
 '<s6>',
 'application',
 'development',
 'associate',
 '</s6>',
 '<s5>']

In [23]:
print(len(docs_full)) # Выводим число записей в наборе данных 

# Выбираем итоговое количество данных для обучающей/проверочной и тестовой выборках
docs = docs_full[0:-10]
docsToTest = docs_full[-10:]

220


In [24]:
print('Число текстов в для тестовой проверки в конце ноутбука:', len(docsToTest)) 
print('Число договоров для обучающей и проверочной выборках:',len(docs)) 
print('Число слов в первом договоре:', len(docs[10]))

Число текстов в для тестовой проверки в конце ноутбука: 10
Число договоров для обучающей и проверочной выборках: 210
Число слов в первом договоре: 93


In [25]:
print('Пример текста обычного:')
print(new_texts[4][:62], '\n')
print('Тот же текст, но представленный ввиде лемм:')
print(docs[4][:10])

Пример текста обычного:
  <s0>Ananya Chavan</s0>   <s6>lecturer</s6>  -  <s5>oracle tu 

Тот же текст, но представленный ввиде лемм:
['<s0>', 'ananya', 'chavan', '</s0>', '<s6>', 'lecturer', '</s6>', '<s5>', 'oracle', 'tutorials']


# Превращение текстов в последовтельность индексов: создание xTrain

In [26]:
import numpy as np # Библиотека для работы с массивами данных
from tensorflow.keras.models import Model, load_model # Импортируем Model, load_model - метод, что загружает предобученную сеть
import re # Имортируем чтобы работать с строками
from tensorflow.keras.preprocessing.text import Tokenizer # Метод, который поволяет работать с текстами и конвертирует их в последовательности (индексов)
# Импорт слоёв нейронных сетей
from tensorflow.keras.layers import Dense, Embedding, Input, concatenate, Activation, MaxPooling1D, Conv1D, BatchNormalization, Dropout, Conv2DTranspose, Conv1DTranspose, Lambda
from tensorflow.keras import backend as K # Импортируем, чтобы высчитать dice_coef(ошибку)
from tensorflow.keras.optimizers import Adam, Adadelta # Импортируем оптимизаторы
from tensorflow.keras import utils # Импортируем для работы с категориальными данными
import tensorflow
from google.colab import files # Импорт для работы с файлами
import matplotlib.pyplot as plt # Импорт для отрисовывания графиков
from gensim.models import word2vec # Импортируем gensim



In [27]:
# lower=True - приводит к нижнему регистру все слова
# char_level=False - говорит токенайзеру не инициализировать отдельно каждую букву как токен  
tokenizer = Tokenizer(lower=True, filters = '', char_level=False)

tokenizer.fit_on_texts(docs_full) # Скармливаем тексты токенайзеру
clean_voc = {}                    # Создаем пустой словарь

for item in tokenizer.word_index.items(): # Превращаем лист в словарь
  clean_voc[item[0]] = item[1]            # Мы меняем местами элеметны кортежа

In [28]:
print('Словарь все слов по их частотноти:') 
print(clean_voc, '\n')
print('Длина словаря:', len(clean_voc))

tag = '<s0>'
print('Индекс тега', tag, ':' ,clean_voc[tag])

Словарь все слов по их частотноти:
{'and': 1, 'the': 2, 'to': 3, 'in': 4, 'of': 5, 'for': 6, 'on': 7, 'work': 8, 'with': 9, 'test': 10, '<s5>': 11, '</s5>': 12, 'indeed': 13, 'be': 14, 'a': 15, 'management': 16, 'experience': 17, 'use': 18, 'project': 19, 'as': 20, 'team': 21, '<s6>': 22, '</s6>': 23, '<s7>': 24, '</s7>': 25, 'skills': 26, '<s8>': 27, '</s8>': 28, 'microsoft': 29, '➢': 30, '1': 31, 'data': 32, 'report': 33, 'engineer': 34, 'years': 35, 'support': 36, 'service': 37, 'tool': 38, 'year': 39, 'oracle': 40, '<s1>': 41, '</s1>': 42, '&': 43, 'process': 44, 'from': 45, 'client': 46, 'application': 47, '<s2>': 48, '</s2>': 49, 'business': 50, 'by': 51, 'technical': 52, 'system': 53, 'sql': 54, 'design': 55, 'than': 56, 'issue': 57, 'it': 58, 'software': 59, 'have': 60, 'email': 61, '*': 62, 'https': 63, '<s3>': 64, '</s3>': 65, '//www': 66, '<s9>': 67, '</s9>': 68, 'me': 69, 'server': 70, 'all': 71, 'sap': 72, 'development': 73, 'develop': 74, 'infosys': 75, 'less': 76, 'knowl

In [29]:
# Превращает текст в последовательность индексов согласно словарю частотности
tok_agreem = tokenizer.texts_to_sequences(docs) # Обучающий выборка в последовтельность индексов

print("Взгляните на фрагмент обучающей выборки:")
print("Исходный текст:              ", docs[0][:20])
print("Тот же текст, но как последовательность индексов: ", tok_agreem[0][:20], '\n')

Взгляните на фрагмент обучающей выборки:
Исходный текст:               ['<s0>', 'abhishek', 'jha', '</s0>', '<s6>', 'application', 'development', 'associate', '</s6>', '<s5>', 'accenture', '</s5>', '<s8>', 'bengaluru', '</s8>', 'karnataka', 'email', 'me', 'on', '<s9>']
Тот же текст, но как последовательность индексов:  [78, 4955, 4956, 79, 22, 47, 73, 246, 23, 11, 492, 12, 27, 158, 28, 81, 61, 69, 7, 67] 



# Создание yTrain

In [30]:
#  Собираем лист индексов и их мультилейбл класссификации
def getXYSamples(tok_agreem, tags_index):
  tags01 = [] # лист тегов
  indices = [] # лист индексов
 
  for agreement in tok_agreem: # Проходимся по каждому договору-листу
    tag_place = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] # создаём вектор [0,0,0,0,0,0] 
    for ex in agreement: # Проходимся по каждому слову из договора
        if ex in tags_index: # Проверяем, если индекс слова соотвествует индексу одного из тегов
          place = np.argwhere(tags_index==ex) # Запоминаем позицию тега в этом листе тегов
          if len(place)!=0: # Проверяем, если тек в самом деле есть
            if place[0][0]<11: # Первые 6 тегов - открывающие теги
              tag_place[place[0][0]] = 1    # устанавливаем значение 1
            else: 
              tag_place[place[0][0] - 11] = 0  # Все остальные теги закрывающие, так что мы меняем в соответсвующем месте в векторе на 0
        else:          
          tags01.append(tag_place.copy()) # Добавляем в наш лист тегов новые вектора мульти-лейбл классификации. 
                                          # В конце концов у нас будет большой лист всех вектор-тегов
          indices.append(ex) # Добавляем индекс-слова в лист индексов

  return indices, tags01

# Получаем лист слов из листа индексов
def reverseIndex(clean_voc, x):
  reverse_word_map = dict(map(reversed, clean_voc.items())) #  Берёт ключ и значение из словаря и меняет их местами
  docs = [reverse_word_map.get(letter) for letter in x] # Положим их в лист
  return docs # Возвращает текст

In [31]:
tags_index = ['<s' + str(i) + '>' for i in range(0, 11)] # Лист открывающих тегов
closetags = ['</s' + str(i) + '>' for i in range(0, 11)] # Лист закрывающих тегов
tags_index.extend(closetags) # конкатенируем все теги

tags_index = np.array([clean_voc[i] for i in tags_index]) # Получаем инексы всех тегов из словря частотности
print('Индексы открывающих тегов:', tags_index[:11])
print('Индексы закрывающих тегов:', tags_index[11:])

Индексы открывающих тегов: [  78   41   48   64  405   11   22   24   27   67 3956]
Индексы закрывающих тегов: [  79   42   49   65  406   12   23   25   28   68 3957]


In [32]:
curTime = time.time()
xData, yData = getXYSamples(tok_agreem,tags_index) # Получаем теги и создаём листы с ними
decoded_text = reverseIndex(clean_voc, xData) # Чтобы создать лист для эмбеддинга, нам нужно превратить лист индексов обратно в лист слов
print('Превращение заняло: ', round(time.time() - curTime, 2), 'с.')

Превращение заняло:  0.75 с.


In [33]:
print('длина xData:', len(xData))
print('длина yData:', len(yData))

print('Посмотрим на исходные слова: ', docs[0][:50])
print('Сдекодированые слова:', decoded_text[:50])
print('Часть xData:     ', xData[:50])
print('Часть yData:     ', yData[:50])

длина xData: 99282
длина yData: 99282
Посмотрим на исходные слова:  ['<s0>', 'abhishek', 'jha', '</s0>', '<s6>', 'application', 'development', 'associate', '</s6>', '<s5>', 'accenture', '</s5>', '<s8>', 'bengaluru', '</s8>', 'karnataka', 'email', 'me', 'on', '<s9>', 'indeed', 'indeed', 'com/r/abhishek', 'jha/10e7a8cb732bc43a', '</s9>', 'to', 'work', 'for', 'an', 'organization', 'which', 'provide', 'me', 'the', 'opportunity', 'to', 'improve', 'my', 'skills', 'and', 'knowledge', 'for', 'my', 'individual', 'and', "company's", 'growth', 'in', 'best', 'possible']
Сдекодированые слова: ['abhishek', 'jha', 'application', 'development', 'associate', 'accenture', 'bengaluru', 'karnataka', 'email', 'me', 'on', 'indeed', 'indeed', 'com/r/abhishek', 'jha/10e7a8cb732bc43a', 'to', 'work', 'for', 'an', 'organization', 'which', 'provide', 'me', 'the', 'opportunity', 'to', 'improve', 'my', 'skills', 'and', 'knowledge', 'for', 'my', 'individual', 'and', "company's", 'growth', 'in', 'best', 'possible', '

# Разделение выборки на окна

In [34]:
# Создание выборки из индексов
def getSetFromIndices(wordIndices, xLen, step): 
  xBatch = [] # Лист, хранящий фрагменты текста
  wordsLen = len(wordIndices) # получаем длину текста
  index = 0 #  стартовый индекс
  
  while (index + xLen <= wordsLen): # Пока сумма индекса и окна меньше или равно чем число слов в выборке
    xBatch.append(wordIndices[index:index+xLen]) # Добавляем X в лист, что содержит наши фрагменты(окна) текста
    index += step # Смещаем наше индекс на наш шаг(step)

  return xBatch #  Лист фрагментов текста

In [35]:
xLen = 256 # Длина каждого окна
step = 30 # шаг 
embeddingSize = 300 # Число измерений вектор-пространства для каждого нашего слова 

In [36]:
#  Генерируем выборки с параметрами наших окон
xTrain = getSetFromIndices(decoded_text, xLen, step) # Последовательность индексов с окном xLen слов
yTrain = getSetFromIndices(yData, xLen, step) # Последовательность индексов с окном xLen от тегов

In [37]:
print('Длина xTrain:', len(xTrain))
print('Длина yTrain:', len(yTrain))
print('Длина примера из xTrain:',len(xTrain[0]))
print('Длина примера из yTrain:',len(yTrain[0]), '\n')
print('Пример xTrain', xTrain[0])
print('Пример yTrain', yTrain[0], '\n')

print('Первый пример xTrain:', xTrain[0][step-5:step+5])
print('Второй пример xTrain:', xTrain[1][:10])

Длина xTrain: 3301
Длина yTrain: 3301
Длина примера из xTrain: 256
Длина примера из yTrain: 256 

Пример xTrain ['abhishek', 'jha', 'application', 'development', 'associate', 'accenture', 'bengaluru', 'karnataka', 'email', 'me', 'on', 'indeed', 'indeed', 'com/r/abhishek', 'jha/10e7a8cb732bc43a', 'to', 'work', 'for', 'an', 'organization', 'which', 'provide', 'me', 'the', 'opportunity', 'to', 'improve', 'my', 'skills', 'and', 'knowledge', 'for', 'my', 'individual', 'and', "company's", 'growth', 'in', 'best', 'possible', 'ways', 'will', 'to', 'relocate', 'to', 'bangalore', 'karnataka', 'work', 'experience', 'application', 'development', 'associate', 'accenture', 'november', '2017', 'to', 'present', 'role', 'currently', 'work', 'on', 'chat', 'bot', 'develop', 'backend', 'oracle', 'peoplesoft', 'query', 'for', 'the', 'bot', 'which', 'will', 'be', 'trigger', 'base', 'on', 'give', 'input', 'also', 'train', 'the', 'bot', 'for', 'different', 'possible', 'utterances', 'both', 'positive', 'and', 

In [38]:
# Превращаем текст в последовательность индексов согласно словарю частотности
tok_agreemTest = tokenizer.texts_to_sequences(docsToTest) # Обучающий текст в индексы

print("Посмотрим на фрагмент тестового текста:")
print("Исходный текст:              ", docsToTest[4][:20])
print("Тот же текст, но как последовательность индексов: ", tok_agreemTest[4][:20], '\n')

xDataTest, yDataTest = getXYSamples(tok_agreemTest,tags_index) # Распознаём теги и создаём лист, что их хранят
decoded_text = reverseIndex(clean_voc, xDataTest) # Чтобы создать лист для эмбеддинга, нам нужно превратить лист индексов обратно в лист слов
print('Длина xDataTest:', len(xDataTest))
print('Длина yDataTest:', len(yDataTest))

print('Сдекодированные текст:', decoded_text[50:80])
print('Часть xDataTest:     ', xDataTest[50:80])
print('Часть yDataTest:     ', yDataTest[50:80])

xLen = 256 # Длина окна
step = 30 # Шаг 
embeddingSize = 300 # Число измерений вектор-пространства для каждого нашего слова 

# Генерируем выборки с определёнными параметрами
xTest = getSetFromIndices(decoded_text, xLen, step) # Последовательность индексов с окном Xlen слов
yTest = getSetFromIndices(yDataTest, xLen, step) # # Последовательность индексов с окном Xlen от тегов

print('Длина xTest:', len(xTest))
print('Длина yTest:', len(yTest))
print('Длина примера xTest:',len(xTest[0]))
print('Длина примера yTrain:',len(yTest[0]), '\n')
print('Пример xTest', xTest[0])
print('Пример yTest', yTest[0], '\n')

print('Первый пример xTest:', xTest[0][step-5:step+5])
print('Второй пример xTest:', xTest[1][:10])

Посмотрим на фрагмент тестового текста:
Исходный текст:               ['<s0>', 'suman', 'biswas', '</s0>', '<s6>', 'sap', 'ui5', 'lead', 'native', 'hana', 'developer', '</s6>', '<s5>', 'royal', 'dutch', 'shell', '</s5>', '<s8>', 'bengaluru', '</s8>']
Тот же текст, но как последовательность индексов:  [78, 8806, 8807, 79, 22, 72, 990, 102, 1959, 755, 182, 23, 11, 8808, 8809, 660, 12, 27, 158, 28] 

Длина xDataTest: 4286
Длина yDataTest: 4286
Сдекодированные текст: ['2003', '2008', '2012', '2012r2', 'exchange', '2003', '2007', '2010', '2013', 'date', '[…]', 'name', 'pawan', 'nag', 'https', '//www', 'indeed', 'com/r/pawan', 'nag/e14493f28cb72022', 'shivam', 'sharma', 'l1', 'analyst', 'in', 'microsoft', 'project', 'hcl', 'technologies', 'ghaziabad', 'uttar']
Часть xDataTest:      [719, 199, 153, 8740, 411, 719, 315, 177, 131, 400, 718, 520, 4892, 4893, 63, 66, 13, 4894, 4895, 4867, 2715, 2203, 251, 4, 29, 19, 922, 147, 2430, 503]
Часть yDataTest:      [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0

# Функции создания моделей
https://youtu.be/wdi8HYhgGj0?t=1574

## Функция, что считает ошибку
https://youtu.be/wdi8HYhgGj0?t=1566

In [39]:
# Функция нужна для точности
def dice_coef(y_true, y_pred):
    return (2. * K.sum(y_true * y_pred) + 1.) / (K.sum(y_true) + K.sum(y_pred) + 1.)

## **Conv1D**

https://youtu.be/wdi8HYhgGj0?t=1635

In [40]:
# Функция, что создаёт линейную свёрточную сеть
def create_Conv1d(xLen, embeddingSize): 
  text_input_layer = Input((xLen,embeddingSize)) 
  text_layer = Conv1D(16, 3, padding='same',activation='relu')(text_input_layer)
  text_layer = Conv1D(16, 3, padding='same',activation='relu')(text_layer)
  text_layer = Conv1D(16, 3,padding='same', activation='relu')(text_layer) 
  text_layer = Conv1D(GENSIMtrainY.shape[-1], 3, padding='same',activation='sigmoid')(text_layer)
  model = Model(text_input_layer, text_layer)
  model.compile(optimizer=Adadelta(),
                    loss='categorical_crossentropy',
                    metrics=[dice_coef])
  return model

## **PSPnet**
https://youtu.be/wdi8HYhgGj0?t=1697

In [41]:
# Функция по созданию сети PSPnet
def create_PSPNet(conv_size = 64, num_classes = 11, input_shape = (30, 300)):
    img_input = Input(input_shape)

    # Блок 1
    x = Conv1D(conv_size, 3, padding='same')(img_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = Conv1D(conv_size, 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x_mp_2 = MaxPooling1D(2)(x)
    x_mp_4 = MaxPooling1D(4)(x)
    x_mp_8 = MaxPooling1D(8)(x)
    x_mp_16 = MaxPooling1D(16)(x)
    x_mp_32 = MaxPooling1D(32)(x)

    x_mp_2 = Conv1D(conv_size, 3, padding='same', activation='relu')(x_mp_2)
    x_mp_2 = Dropout(0.5)(x_mp_2)

    x_mp_4 = Conv1D(conv_size, 3, padding='same', activation='relu')(x_mp_4)
    x_mp_4 = Dropout(0.5)(x_mp_4)

    x_mp_8 = Conv1D(conv_size, 3, padding='same', activation='relu')(x_mp_8)
    x_mp_8 = Dropout(0.5)(x_mp_8)

    x_mp_16 = Conv1D(conv_size, 3, padding='same', activation='relu')(x_mp_16)
    x_mp_16 = Dropout(0.5)(x_mp_16)

    x_mp_32 = Conv1D(conv_size, 3, padding='same', activation='relu')(x_mp_32)
    x_mp_32 = Dropout(0.5)(x_mp_32) 

    x_mp_2 = Conv1DTranspose(conv_size, 2, strides=2)(x_mp_2)
    x_mp_2 = Activation('relu')(x_mp_2)

    x_mp_4 = Conv1DTranspose(conv_size, 4, strides=4)(x_mp_4)
    x_mp_4 = Activation('relu')(x_mp_4)

    x_mp_8 = Conv1DTranspose(conv_size, 8, strides=8)(x_mp_8)
    x_mp_8 = Activation('relu')(x_mp_8)

    x_mp_16 = Conv1DTranspose(conv_size, 16, strides=16)(x_mp_16)
    x_mp_16 = Activation('relu')(x_mp_16)

    x_mp_32 = Conv1DTranspose(conv_size, 32, strides=32)(x_mp_32)
    x_mp_32 = Activation('relu')(x_mp_32)

    fin = concatenate([img_input, x_mp_2, x_mp_4, x_mp_8, x_mp_16, x_mp_32])    
    fin = Conv1D(conv_size, 3, padding='same')(fin)
    fin = BatchNormalization()(fin)
    fin = Activation('relu')(fin)
    fin = Conv1D(conv_size, 3, padding='same')(fin)
    fin = BatchNormalization()(fin)
    fin = Activation('relu')(fin)

    fin = Conv1D(num_classes, 3, activation='sigmoid', padding='same')(fin)

    model = Model(img_input, fin)
    model.compile(optimizer=Adadelta(lr=0.001),
                  loss='categorical_crossentropy',
                  metrics=[dice_coef])
    
    return model

##**UNET**
https://youtu.be/wdi8HYhgGj0?t=1769

In [42]:
# Функция по созданию сети UNET
def create_unet(k = 1, num_classes = 11, input_shape= (30, 300)):
    img_input = Input(input_shape) 

    # Блок 1
    x = Conv1D(64 * k , 3, padding='same')(img_input) 
    x = BatchNormalization()(x) 
    x = Activation('relu')(x)

    x = Conv1D(64 * k, 3, padding='same')(x)  
    x = BatchNormalization()(x)     
    block_1_out = Activation('relu')(x) 

    # Блок 2
    x = MaxPooling1D()(block_1_out)
    x = Conv1D(128 * k, 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)  

    x = Conv1D(128 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    block_2_out = Activation('relu')(x)

    # Блок 3
    x = MaxPooling1D()(block_2_out)
    x = Conv1D(256 * k, 3, padding='same')(x)
    x = BatchNormalization()(x)               
    x = Activation('relu')(x)                     

    x = Conv1D(256 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(256 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    block_3_out = Activation('relu')(x)

    # Блок 4
    x = MaxPooling1D()(block_3_out)
    x = Conv1D(512 * k, 3, padding='same')(x)
    x = BatchNormalization()(x) 
    x = Activation('relu')(x)

    x = Conv1D(512 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(512 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)      
    block_4_out = Activation('relu')(x)
    x = block_4_out 

    # UP 2
    x = Conv1DTranspose(256 * k, kernel_size=3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x) 

    x = concatenate([x, block_3_out]) 
    x = Conv1D(256 * k , 3, padding='same')(x) 
    x = BatchNormalization()(x) 
    x = Activation('relu')(x)

    x = Conv1D(256 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # UP 3
    x = Conv1DTranspose(128 * k, kernel_size=3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x) 

    x = concatenate([x, block_2_out])
    x = Conv1D(128 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(128 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # UP 4
    x = Conv1DTranspose(64 * k, kernel_size=3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = concatenate([x, block_1_out])
    x = Conv1D(64 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(64 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(num_classes, 3, activation='sigmoid', padding='same')(x)

    model = Model(img_input, x) 

    model = Model(img_input, x)
    model.compile(optimizer=Adam(0.0025), 
                  loss='categorical_crossentropy',
                  metrics=[dice_coef])
  
    return model

# Создание xTrain и  yTrain используя Word2Vec GENSIM

https://youtu.be/wdi8HYhgGj0?t=1830


In [43]:
def pad_zeros(phrase, xLen = 256): # Функция для дополнением нулями
  while len(phrase) < xLen: # Пока сущестующий размер меньше чем требуемый 
    phrase.append([0] * embeddingSize) # Продолжаем добавлять элементы(нули) в фразу(zero-pad)  
  if len(phrase) > xLen: # В противном случае, если фраза больше чем  требуемый размер
    phrase = phrase[:xLen] # Берём только требуемое число элементов
  return phrase # Возвращаем обработанную фразу

# Создаём выборку
def getSets(model, senI, tagI):
  xVector = [] # Лист будет содержать ембеддинг представление на каждый индекс
  tmp = [] # Временный список
  for text in senI: # Проходимся по каждому лист-тексту
    tmp=[]
    for word in text: # Проходимся по кажому слову лист-текста
      try: # Если слово есть в модели (словарь не слишком мал), мы не получим ошибку
        tmp.append(model[word])
      except: # В противном случае пропустим слово
        pass

    xVector.append(pad_zeros(tmp, xLen))
  temp = np.array(xVector)
  return np.array(xVector, dtype = np.float32), np.array(tagI)

In [44]:
# Подаём лист листов слов в word2vec для обучающей выборке
# size = embeddingSize - Размер ембеддинга
# window = 10 -  минимальное расстояния между эмбеддинг словами
# min_count = 1 - Игнорируем все слова с частотой меньше чем 1
# workers = 10 -  Число потоков на обучение эмбеддинга
# iter = 20 -  Число эпох на обучение эмбеддинга
# max_vocab_size = 1e5 -  Число слов в "словарном запасе" word2vec
curTime = time.time()

modelGENSIM = word2vec.Word2Vec(xTrain + xTest, size = embeddingSize, window = 10, min_count = 1, workers = 10, iter = 10, max_vocab_size = 1e5)
print('Обучение GENSIM модели заняло: ', round(time.time() - curTime, 2), 'с.')

Обучение GENSIM модели заняло:  17.47 с.


In [46]:
modelGENSIM['education'] # Пример превращения слова в вектор при помощи GENSIM

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 3.2154312 , -0.5298406 ,  2.155577  , -0.4374456 ,  0.67386866,
        0.7269224 ,  1.4426969 ,  1.9769449 ,  1.1434984 ,  1.3292294 ,
       -4.0356855 , -0.21189646, -0.10808229,  1.6741306 , -1.094256  ,
        1.5311981 ,  1.5448023 ,  1.9651257 , -0.5191166 ,  1.421021  ,
       -1.6495837 , -2.9194953 ,  1.1780385 ,  2.0927384 ,  2.5980332 ,
       -1.2511398 ,  3.0788505 ,  0.3801587 , -1.5925753 ,  0.3489633 ,
       -0.04976106,  2.056907  ,  0.9842874 , -1.6463654 ,  0.6465096 ,
       -1.0147517 , -2.9270062 , -1.7677222 , -1.0815783 , -0.5335973 ,
       -1.6391389 ,  0.83242804,  0.8930105 ,  1.4093379 , -0.62933856,
       -0.5475573 , -0.04790112,  2.095309  ,  2.454906  ,  0.30066928,
       -2.5687857 , -2.2330005 , -1.197821  ,  0.2989786 , -2.3574698 ,
        1.2166109 , -0.9348499 , -1.1456035 , -1.2778476 ,  3.6232152 ,
       -0.46689966, -2.5595548 ,  0.2635063 , -2.0429075 , -3.520094  ,
        0.12303239, -0.55002004, -0.84671247,  2.057597  ,  2.29

In [47]:
GENSIMtrainX, GENSIMtrainY = getSets(modelGENSIM, xTrain, yTrain)
GENSIMtestX, GENSIMtestY = getSets(modelGENSIM, xTest, yTest)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  app.launch_new_instance()


In [48]:
print('Размерности xTrain:', GENSIMtrainX.shape)
print('Размерности yTrain:', GENSIMtrainY.shape)

Размерности xTrain: (3301, 256, 300)
Размерности yTrain: (3301, 256, 11)


# Train the networks


## **Linear Conv1d network**


In [49]:
model_conv1d = create_Conv1d(xLen, embeddingSize) # Создаем модель
model_conv1d.summary() # Выводим саммери

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 300)]        0         
_________________________________________________________________
conv1d (Conv1D)              (None, 256, 16)           14416     
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 256, 16)           784       
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 256, 16)           784       
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 256, 11)           539       
Total params: 16,523
Trainable params: 16,523
Non-trainable params: 0
_________________________________________________________________


In [50]:
history = model_conv1d.fit(GENSIMtrainX, GENSIMtrainY, epochs=50, batch_size=200, validation_data=(GENSIMtestX, GENSIMtestY)) # Обучаем модель на сверточной сети

Epoch 1/50
17/17 [==============================] - 18s 54ms/step - loss: 0.3847 - dice_coef: 0.0238 - val_loss: 0.2592 - val_dice_coef: 0.0176
Epoch 2/50
17/17 [==============================] - 1s 38ms/step - loss: 0.3845 - dice_coef: 0.0238 - val_loss: 0.2590 - val_dice_coef: 0.0177
Epoch 3/50
17/17 [==============================] - 1s 38ms/step - loss: 0.3842 - dice_coef: 0.0238 - val_loss: 0.2589 - val_dice_coef: 0.0177
Epoch 4/50
17/17 [==============================] - 1s 37ms/step - loss: 0.3839 - dice_coef: 0.0239 - val_loss: 0.2587 - val_dice_coef: 0.0177
Epoch 5/50
17/17 [==============================] - 1s 38ms/step - loss: 0.3836 - dice_coef: 0.0238 - val_loss: 0.2585 - val_dice_coef: 0.0177
Epoch 6/50
17/17 [==============================] - 1s 38ms/step - loss: 0.3833 - dice_coef: 0.0239 - val_loss: 0.2584 - val_dice_coef: 0.0177
Epoch 7/50
17/17 [==============================] - 1s 38ms/step - loss: 0.3830 - dice_coef: 0.0239 - val_loss: 0.2582 - val_dice_coef: 0.017

## **PSPnet**


### Стандартная архитектура

In [51]:
model_b_PSPnet = create_PSPNet(input_shape=(xLen, embeddingSize))
model_b_PSPnet.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 256, 300)]   0                                            
__________________________________________________________________________________________________
conv1d_4 (Conv1D)               (None, 256, 64)      57664       input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 256, 64)      256         conv1d_4[0][0]                   
__________________________________________________________________________________________________
activation (Activation)         (None, 256, 64)      0           batch_normalization[0][0]        
____________________________________________________________________________________________

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [52]:
history = model_b_PSPnet.fit(GENSIMtrainX, GENSIMtrainY, validation_data = (GENSIMtestX, GENSIMtestY), epochs=50, batch_size=64)

Epoch 1/50
52/52 [==============================] - 4s 43ms/step - loss: 0.3773 - dice_coef: 0.0271 - val_loss: 0.2458 - val_dice_coef: 0.0190
Epoch 2/50
52/52 [==============================] - 1s 28ms/step - loss: 0.3743 - dice_coef: 0.0274 - val_loss: 0.2482 - val_dice_coef: 0.0190
Epoch 3/50
52/52 [==============================] - 1s 28ms/step - loss: 0.3711 - dice_coef: 0.0276 - val_loss: 0.2496 - val_dice_coef: 0.0192
Epoch 4/50
52/52 [==============================] - 1s 28ms/step - loss: 0.3681 - dice_coef: 0.0279 - val_loss: 0.2500 - val_dice_coef: 0.0193
Epoch 5/50
52/52 [==============================] - 1s 27ms/step - loss: 0.3650 - dice_coef: 0.0283 - val_loss: 0.2494 - val_dice_coef: 0.0195
Epoch 6/50
52/52 [==============================] - 1s 27ms/step - loss: 0.3618 - dice_coef: 0.0286 - val_loss: 0.2480 - val_dice_coef: 0.0198
Epoch 7/50
52/52 [==============================] - 1s 27ms/step - loss: 0.3585 - dice_coef: 0.0289 - val_loss: 0.2461 - val_dice_coef: 0.0200

### 256 нейронов в свёрточном ядре

In [53]:
model_b_PSPnet = create_PSPNet(conv_size = 256, input_shape=(xLen, embeddingSize))
model_b_PSPnet.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 256, 300)]   0                                            
__________________________________________________________________________________________________
conv1d_14 (Conv1D)              (None, 256, 256)     230656      input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_4 (BatchNor (None, 256, 256)     1024        conv1d_14[0][0]                  
__________________________________________________________________________________________________
activation_9 (Activation)       (None, 256, 256)     0           batch_normalization_4[0][0]      
____________________________________________________________________________________________

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [54]:
history = model_b_PSPnet.fit(GENSIMtrainX, GENSIMtrainY, validation_data = (GENSIMtestX, GENSIMtestY), epochs=50, batch_size=64)

Epoch 1/50
52/52 [==============================] - 7s 87ms/step - loss: 0.4489 - dice_coef: 0.0216 - val_loss: 0.2640 - val_dice_coef: 0.0159
Epoch 2/50
52/52 [==============================] - 4s 72ms/step - loss: 0.4319 - dice_coef: 0.0228 - val_loss: 0.2608 - val_dice_coef: 0.0164
Epoch 3/50
52/52 [==============================] - 4s 72ms/step - loss: 0.4152 - dice_coef: 0.0242 - val_loss: 0.2563 - val_dice_coef: 0.0171
Epoch 4/50
52/52 [==============================] - 4s 72ms/step - loss: 0.3996 - dice_coef: 0.0257 - val_loss: 0.2496 - val_dice_coef: 0.0180
Epoch 5/50
52/52 [==============================] - 4s 72ms/step - loss: 0.3849 - dice_coef: 0.0271 - val_loss: 0.2414 - val_dice_coef: 0.0191
Epoch 6/50
52/52 [==============================] - 4s 72ms/step - loss: 0.3706 - dice_coef: 0.0286 - val_loss: 0.2324 - val_dice_coef: 0.0202
Epoch 7/50
52/52 [==============================] - 4s 73ms/step - loss: 0.3572 - dice_coef: 0.0301 - val_loss: 0.2232 - val_dice_coef: 0.0213

### 512 нейронов в свёрточном ядре

In [55]:
model_b_PSPnet = create_PSPNet(conv_size = 512, input_shape=(xLen, embeddingSize))
model_b_PSPnet.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 256, 300)]   0                                            
__________________________________________________________________________________________________
conv1d_24 (Conv1D)              (None, 256, 512)     461312      input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_8 (BatchNor (None, 256, 512)     2048        conv1d_24[0][0]                  
__________________________________________________________________________________________________
activation_18 (Activation)      (None, 256, 512)     0           batch_normalization_8[0][0]      
____________________________________________________________________________________________

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [56]:
history = model_b_PSPnet.fit(GENSIMtrainX, GENSIMtrainY, validation_data = (GENSIMtestX, GENSIMtestY), epochs=50, batch_size=64)

Epoch 1/50
52/52 [==============================] - 13s 197ms/step - loss: 0.3717 - dice_coef: 0.0281 - val_loss: 0.2417 - val_dice_coef: 0.0183
Epoch 2/50
52/52 [==============================] - 9s 176ms/step - loss: 0.3469 - dice_coef: 0.0301 - val_loss: 0.2290 - val_dice_coef: 0.0197
Epoch 3/50
52/52 [==============================] - 9s 176ms/step - loss: 0.3264 - dice_coef: 0.0321 - val_loss: 0.2155 - val_dice_coef: 0.0213
Epoch 4/50
52/52 [==============================] - 9s 176ms/step - loss: 0.3091 - dice_coef: 0.0339 - val_loss: 0.2024 - val_dice_coef: 0.0229
Epoch 5/50
52/52 [==============================] - 9s 176ms/step - loss: 0.2942 - dice_coef: 0.0356 - val_loss: 0.1912 - val_dice_coef: 0.0244
Epoch 6/50
52/52 [==============================] - 9s 176ms/step - loss: 0.2823 - dice_coef: 0.0373 - val_loss: 0.1816 - val_dice_coef: 0.0257
Epoch 7/50
52/52 [==============================] - 9s 176ms/step - loss: 0.2719 - dice_coef: 0.0388 - val_loss: 0.1735 - val_dice_coef

## **UNET**


### Стандартная архитектура

In [57]:
model_b_UNET = create_unet(input_shape=(xLen, embeddingSize))
model_b_UNET.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 256, 300)]   0                                            
__________________________________________________________________________________________________
conv1d_34 (Conv1D)              (None, 256, 64)      57664       input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_12 (BatchNo (None, 256, 64)      256         conv1d_34[0][0]                  
__________________________________________________________________________________________________
activation_27 (Activation)      (None, 256, 64)      0           batch_normalization_12[0][0]     
____________________________________________________________________________________________

In [58]:
history = model_b_UNET.fit(GENSIMtrainX, GENSIMtrainY, validation_data = (GENSIMtestX, GENSIMtestY), epochs=50, batch_size=64) # Feed the images divided among 2 classes to the model

Epoch 1/50
52/52 [==============================] - 8s 63ms/step - loss: 0.2154 - dice_coef: 0.1516 - val_loss: 1.4593 - val_dice_coef: 0.0034
Epoch 2/50
52/52 [==============================] - 3s 48ms/step - loss: 0.2238 - dice_coef: 0.1888 - val_loss: 3.8799 - val_dice_coef: 0.0028
Epoch 3/50
52/52 [==============================] - 2s 48ms/step - loss: 0.2322 - dice_coef: 0.1955 - val_loss: 0.3153 - val_dice_coef: 0.0134
Epoch 4/50
52/52 [==============================] - 3s 48ms/step - loss: 0.2937 - dice_coef: 0.1480 - val_loss: 0.2518 - val_dice_coef: 0.0741
Epoch 5/50
52/52 [==============================] - 3s 48ms/step - loss: 0.3432 - dice_coef: 0.1202 - val_loss: 0.3542 - val_dice_coef: 0.0225
Epoch 6/50
52/52 [==============================] - 2s 48ms/step - loss: 0.3410 - dice_coef: 0.1227 - val_loss: 0.2456 - val_dice_coef: 0.0285
Epoch 7/50
52/52 [==============================] - 3s 48ms/step - loss: 0.3383 - dice_coef: 0.1239 - val_loss: 0.2436 - val_dice_coef: 0.0753

# **Проверяем работу сети**

In [69]:
def recognizeSet(XX, YY, model, tags, length, value):
  correct_list = np.array([0] * 11) #Инициализируем массив правильных ответов в нули (сколько раз правильно определили класс)
  incorrect_list =  np.array([0] * 11)  #Инициализируем массив неправильных ответов в нули  (сколько раз неправильно определили класс)
  XX_array = XX
  YY_array = YY
  pred = model.predict(XX_array)
  pred[pred < value] = 0
  pred[pred > value] = 1

  for element in range(YY_array.shape[0]): # Проходим по всем примерам в батче
    for word in range(YY_array.shape[1]):  # Проходим по каждому слову
      for category in range(YY_array.shape[2]): # Проходим по каждой категории в слове
        if pred[element][word][category] == YY_array[element][word][category]: # Если предсказанное значение совпадает с истинным:
          correct_list[category] += 1 # Отмечаем, что мы правильно предсказали класс объекта
        else:  # Если предсказанное значение НЕ совпадает с истенным:
          incorrect_list[category] += 1 # Отмечаем, что мы не правильно предсказали класс объекта
      
  # Итоговая точность для каждого класса = кол. 100% * правильных/(кол. неправильных + кол. правильных)
  for i in range(11):
   print("Сеть распознала категорию  '{}' с точностью в {}%".format(tags[i], round(100*correct_list[i]/(correct_list[i] + incorrect_list[i]), 2)))
  total = round(100*np.mean(correct_list/(correct_list + incorrect_list)),2) 
  print("Средняя точность {}%".format(total))

In [70]:
print('Размер xTrain:', GENSIMtestX.shape)
print('Размер yTrain:', GENSIMtestY.shape)

Размер xTrain: (135, 256, 300)
Размер yTrain: (135, 256, 11)


In [71]:
tags = ['S0', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10']

In [72]:
recognizeSet(GENSIMtestX, GENSIMtestY, model_conv1d, tags, xLen, 0.5) # Проверяем conv1d архитектуру

Сеть распознала категорию  'S0' с точностью в 25.7%
Сеть распознала категорию  'S1' с точностью в 32.99%
Сеть распознала категорию  'S2' с точностью в 7.0%
Сеть распознала категорию  'S3' с точностью в 96.73%
Сеть распознала категорию  'S4' с точностью в 18.98%
Сеть распознала категорию  'S5' с точностью в 38.02%
Сеть распознала категорию  'S6' с точностью в 14.87%
Сеть распознала категорию  'S7' с точностью в 59.69%
Сеть распознала категорию  'S8' с точностью в 2.88%
Сеть распознала категорию  'S9' с точностью в 39.61%
Сеть распознала категорию  'S10' с точностью в 85.46%
Средняя точность 38.36%


In [73]:
recognizeSet(GENSIMtestX, GENSIMtestY, model_b_PSPnet, tags, xLen, 0.5) # Проверяем PSPnet архитектуру

Сеть распознала категорию  'S0' с точностью в 94.57%
Сеть распознала категорию  'S1' с точностью в 91.79%
Сеть распознала категорию  'S2' с точностью в 98.58%
Сеть распознала категорию  'S3' с точностью в 98.5%
Сеть распознала категорию  'S4' с точностью в 98.56%
Сеть распознала категорию  'S5' с точностью в 92.83%
Сеть распознала категорию  'S6' с точностью в 95.95%
Сеть распознала категорию  'S7' с точностью в 92.29%
Сеть распознала категорию  'S8' с точностью в 99.77%
Сеть распознала категорию  'S9' с точностью в 99.67%
Сеть распознала категорию  'S10' с точностью в 99.55%
Средняя точность 96.55%


In [74]:
recognizeSet(GENSIMtestX, GENSIMtestY, model_b_UNET, tags, xLen, 0.1) # Проверяем UNET архитектуру

Сеть распознала категорию  'S0' с точностью в 99.61%
Сеть распознала категорию  'S1' с точностью в 99.2%
Сеть распознала категорию  'S2' с точностью в 98.63%
Сеть распознала категорию  'S3' с точностью в 99.89%
Сеть распознала категорию  'S4' с точностью в 99.97%
Сеть распознала категорию  'S5' с точностью в 99.07%
Сеть распознала категорию  'S6' с точностью в 98.72%
Сеть распознала категорию  'S7' с точностью в 95.87%
Сеть распознала категорию  'S8' с точностью в 99.49%
Сеть распознала категорию  'S9' с точностью в 99.45%
Сеть распознала категорию  'S10' с точностью в 100.0%
Средняя точность 99.08%
